# Домашнее задание. Нейросетевая классификация текстов

В этом домашнем задании вам предстоит самостоятельно решить задачу классификации текстов. Вы будете классифицировать тксты писателей по авторству. Всего 8 разных авторов.

Скачаем датасет:

In [ ]:
!wget -O train_authors.csv https://raw.githubusercontent.com/MSUcourses/Data-Analysis-with-Python/main/Deep%20Learning/Files/train_authors.csv

--2024-04-07 19:25:20--  https://raw.githubusercontent.com/MSUcourses/Data-Analysis-with-Python/main/Deep%20Learning/Files/train_authors.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4883974 (4.7M) [text/plain]
Saving to: ‘train_authors.csv’

train_authors.csv   100%[===================>]   4.66M  --.-KB/s    in 0.06s   

2024-04-07 19:25:20 (78.0 MB/s) - ‘train_authors.csv’ saved [4883974/4883974]



In [ ]:
!wget -O test_authors.csv https://raw.githubusercontent.com/MSUcourses/Data-Analysis-with-Python/main/Deep%20Learning/Files/test_authors.csv

--2024-04-07 19:25:20--  https://raw.githubusercontent.com/MSUcourses/Data-Analysis-with-Python/main/Deep%20Learning/Files/test_authors.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4871403 (4.6M) [text/plain]
Saving to: ‘test_authors.csv’

test_authors.csv    100%[===================>]   4.65M  --.-KB/s    in 0.05s   

2024-04-07 19:25:20 (88.3 MB/s) - ‘test_authors.csv’ saved [4871403/4871403]



Импортируем необходимые библиотеки:

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import numpy as np
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import nltk
from tqdm import tqdm_notebook
import torch.nn as nn                                                             # тут все блоки нейронных сетей, слои
import torch.nn.functional as F

from collections import Counter
from typing import List
import string

import seaborn
seaborn.set(palette='summer')

# скачиваем нужный пакет данных для работы библиотеки nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Подготовка данных

Загрузим даатсет и посмотрим на данные:

In [ ]:
# Загрузим датасет
import pandas as pd
train_data = pd.read_csv('train_authors.csv')
train_data.head()

,text,label
0,"-Да, я поторопился. Капитан, примите мои извин...",Pratchett
1,-Похороны по первому разряду! Довольно благоро...,Remark
2,"Третий округ штата Мэн настолько велик, что ег...",King
3,В мире существуют миллиарды и миллиарды богов....,Pratchett
4,Особенность историографии киевского периода со...,Akunin


Здесь поле 'text' — это текст, а 'label' — автор текста. 'label' — это целевая переменная, которую мы будем предсказывать. Обратите внимания, что в тестовых данных столбца 'label' нет, вам нужно будет предсказать его значения и отправить в качестве ответа на Яндекс.Контест.

In [ ]:
test_data = pd.read_csv('test_authors.csv')
test_data.head()

,text
0,"-Да, я поторопился. Капитан, примите мои извин..."
1,-Похороны по первому разряду! Довольно благоро...
2,"Третий округ штата Мэн настолько велик, что ег..."
3,В мире существуют миллиарды и миллиарды богов....
4,Особенность историографии киевского периода со...


Заведем словарь соответствия имени автора его номеру:

In [ ]:
writers = ['Akunin', 'Bulychev', 'Chehov', 'Dostoevsky', 'Gogol', 'King',
       'Pratchett', 'Remark']
writers_to_label = {writer: i for i, writer in enumerate(writers)}
label_to_writers = {i: writer for i, writer in enumerate(writers)}

Создадим словарь dataset, который будет устроен так же, как переменная dataset в ноутбуке занятия. В тестовой части (dataset['test']) зададим для удобства всем текстам label=0.

In [ ]:
dataset = {}

dataset['train'] = [{'text':text, 'label':writers_to_label[label]} \
              for text, label in zip(np.array(train_data['text']), np.array(train_data['label']))]
dataset['test'] = [{'text':text, 'label': 0} \
              for text in np.array(test_data['text'])]

In [ ]:
# библиотека для работы со строками. С ее помощью мы удалим знаки препинания
import string
# счетчик для создания счетчика слов
from collections import Counter
# библиотека, с помощью которой разобьем текст на токены
import nltk
from nltk.tokenize import word_tokenize

# скачиваем нужный пакет данных для работы библиотеки nltk
nltk.download('punkt')

# функция для обработки предложения
def process_and_tokenize_text(text):
    # приводим к нижнему регистру и убираем пунктуацию
    prccessed_text = text.lower().translate(
        str.maketrans('', '', string.punctuation)
    )
    # токенизируем текст
    tokens = word_tokenize(prccessed_text)
    return tokens

# массивы для хранения токенизированных тренировочных и тестовых данных
train_data = []
test_data = []

# пустой счетчик слов
words = Counter()

# проходимся по текстам обучающего датасета и предобрабатываем их
for example in tqdm_notebook(dataset['train']):
    text = example['text']
    label = example['label']
    text_processed = process_and_tokenize_text(text)
    train_data.append((text_processed, label))

    # увеличиваем счетчик каждого слова в словаре
    for word in text_processed:
        words[word] += 1

# проходимся по текстам тестового датасета и предобрабатываем их
for example in tqdm_notebook(dataset['test']):
    text = example['text']
    label = example['label']
    text_processed = process_and_tokenize_text(text)
    test_data.append((text_processed, label))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
<ipython-input-9-ccb0c4e80201>:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for example in tqdm_notebook(dataset['train']):


  0%|          | 0/1651 [00:00<?, ?it/s]

<ipython-input-9-ccb0c4e80201>:41: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for example in tqdm_notebook(dataset['test']):


  0%|          | 0/1651 [00:00<?, ?it/s]

In [ ]:
# создаем словарь как множество слов
vocab = set(['<unk>', '<bos>', '<eos>', '<pad>'])
# будем добавлять в словарь только те слова, которые
# встречались в обучающих данных не менее 25 раз
#counter_threshold = 5
#counter_threshold = 3

counter_threshold = 2

# пополняем словарь словами из words
for char, cnt in words.items():
    if cnt > counter_threshold:
      vocab.add(char)

In [ ]:
len(vocab)

18929

In [ ]:
word2ind = {char: i for i, char in enumerate(vocab)}
ind2word = {i: char for char, i in word2ind.items()}

In [ ]:
def collate_fn_with_padding(input_batch, max_len=256):
    # input_batch — батч пар (токенизированный текст, label)
    texts = [x[0] for x in input_batch]
    labels = [x[1] for x in input_batch]
    # для каждого элемента батча получаем длину токенизированного текста в токенах
    seq_lens = [len(x) for x in texts]
    # определяем максимальную длину элементов в текущем батче
    max_seq_len = min(max(seq_lens), max_len)

    # проходимся по элементам батча и заменяем токены на их индексы из словаря
    # также последовательности короче max_seq_len дополняем токенами <pad>
    processed_texts = []
    for text, label in zip(texts, labels):
        text = text[:max_seq_len]
        text = [word2ind[x] if x in vocab else word2ind['<unk>'] for x in text]
        for _ in range(max_seq_len - len(text)):
            text.append(word2ind['<pad>'])

        processed_texts.append(text)

    # переводим элементы батча в форматы тензоров
    processed_texts = torch.LongTensor(processed_texts).to(device)
    labels = torch.LongTensor(labels).to(device)

    # собираем батч
    processed_batch = {
        'input_ids': processed_texts,
        'label': labels
    }

    return processed_batch

In [ ]:
len(train_data)

1651

In [ ]:
# поделим тестовые данные на val и test
np.random.seed(42)
val_indices = np.random.choice(np.arange(len(train_data)), 330)
train_indices = [x for x in range(len(train_data)) if x not in val_indices]
val_data = [train_data[i] for i in val_indices]
train_data = [train_data[i] for i in train_indices]

# заводим даталоадеры
#batch_size = 32
batch_size = 128
train_dataloader = DataLoader(
    train_data, shuffle=True, collate_fn=collate_fn_with_padding, batch_size=batch_size)

val_dataloader = DataLoader(
    val_data, shuffle=False, collate_fn=collate_fn_with_padding, batch_size=batch_size)

test_dataloader = DataLoader(
    test_data, shuffle=False, collate_fn=collate_fn_with_padding, batch_size=batch_size)

* **Изменение вида агрреграции выхода RNN**. В семинаре мы пробовали аггрегировать выходы RNN с помощью max, mean и получения последнего выхода. Можно попробовать другие варианты. Например, конкатенировать результат агрегации и эмбеддинг с последнего токена.

In [ ]:
class SimpleRNN(nn.Module):
    def __init__(
        self, hidden_dim, vocab_size, num_classes,
        aggregation_type: str = 'last'
        ):
        super().__init__()
        # слой эмбеддинга
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        # рекуррентный слой
        self.rnn = nn.RNN(hidden_dim, hidden_dim, num_layers=1, batch_first=True)
        #self.gru = nn.GRU(hidden_dim, hidden_dim, num_layers=1, batch_first=True)
        # два полносвязных слоя
        self.fc1 = nn.Linear(hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

        # слой дропаут
        #self.dropout = nn.Dropout(p=0.1)
        self.dropout1 = nn.Dropout(p=0.1)
        #self.dropout2 = nn.Dropout(p=0.2)


        # каким образом выходы RNN-слоя для разных моментов времени будут
        # агрегироваться перед подачей дальше на вход полносвязному слою
        self.aggregation_type = aggregation_type

    def forward(self, input_batch) -> torch.Tensor:
        embeddings = self.embedding(input_batch)  # [batch_size, seq_len, hidden_dim]
        output, _ = self.rnn(embeddings)          # [batch_size, seq_len, hidden_dim]
        #print("embeddings_shape", embeddings.shape)
        #print("emb_-1", embeddings[-1].shape)
        #print("probably_needed_emb", embeddings[:, -1, :].shape)
        #last_emb = embeddings[:, -1, :]
        if self.aggregation_type == 'max':
            output = output.max(dim=1)[0]
            #print("max", output.shape)         #[batch_size, hidden_dim]
            #output = torch.cat((output, last_emb), dim = 1)
        elif self.aggregation_type == 'mean':
            output = output.mean(dim=1)
            #print("mean", output.shape)
            #output = torch.cat((output, last_emb), dim = 1)       #[batch_size, hidden_dim]
        elif self.aggregation_type == 'last':
            output = output[:, -1, :]
            #print("last", output.shape)
            #output = torch.cat((output, last_emb), dim = 1)
        else:
            raise ValueError("Invalid aggregation_type")

        output = F.tanh(output)
        output = F.tanh(self.dropout1(self.fc1(output)))   # [batch_size, hidden_dim]
        #output = F.tanh(self.fc1(output))
        output = self.fc2(output)                        # [batch_size, num_classes]

        return output

In [ ]:
def evaluate(model, dataloader):
    """
    Calculate accuracy on data from dataloader.
    """

    predictions = []
    target = []
    with torch.no_grad():
        for batch in tqdm_notebook(dataloader,
                                   desc=f'Evaluating'):
            logits = model(batch['input_ids'])
            predictions.append(logits.argmax(dim=1))
            target.append(batch['label'])

    predictions = torch.cat(predictions)
    target = torch.cat(target)
    accuracy = (predictions == target).float().mean().item()

    return accuracy

In [ ]:
def train(model, optimizer, criterion, num_epoch=5, eval_steps=100):

    losses = []
    accs_train = []
    accs_val = []

    for epoch in range(num_epoch):
        epoch_losses = []
        model.train()
        for i, batch in enumerate(tqdm_notebook(train_dataloader,
                                                desc=f'Training epoch {epoch}:')):
            optimizer.zero_grad()
            logits = model(batch['input_ids'])
            loss = criterion(logits, batch['label'])
            loss.backward()
            optimizer.step()

            epoch_losses.append(loss.item())
            if i % eval_steps == 0:
                model.eval()
                accs_train.append(evaluate(model, train_dataloader))
                accs_val.append(evaluate(model, val_dataloader))
                model.train()

        losses.append(sum(epoch_losses) / len(epoch_losses))

    return losses, accs_train, accs_val

In [ ]:
num_epoch = 30
eval_steps = len(train_dataloader) // 2
#hidden_dim = 1280
#hidden_dim = 256
#hidden_dim = 512
#hidden_dim = 1536
#hidden_dim = 768
#hidden_dim = 900
hidden_dim = 1024
losses_type = {}
accs_train_type = {}
accs_val_type = {}

# for aggregation_type in ['max', 'mean', 'last']:
#     print(f"Starting training for {aggregation_type}")
#     losses = []
#     acc = []

#     model = SimpleRNN(hidden_dim=hidden_dim,
#                       vocab_size=len(vocab),
#                       num_classes=8,
#                       aggregation_type=aggregation_type
#                       ).to(device)
#     criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
#     optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)

#     losses, accs_train, accs_val = train(model,
#                                          optimizer,
#                                          criterion,
#                                          num_epoch=num_epoch,
#                                          eval_steps=len(train_dataloader) // 2)

#     losses_type[aggregation_type] = losses
#     accs_train_type[aggregation_type] = accs_train
#     accs_val_type[aggregation_type] = accs_val

model = SimpleRNN(hidden_dim=hidden_dim,
                      vocab_size=len(vocab),
                      num_classes=8,
                      aggregation_type="mean"
                      ).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
losses, accs_train, accs_val = train(model,
                                         optimizer,
                                         criterion,
                                         num_epoch=num_epoch,
                                         eval_steps=len(train_dataloader) // 2)


<ipython-input-18-48d3f6fda81d>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, batch in enumerate(tqdm_notebook(train_dataloader,


Training epoch 0::   0%|          | 0/11 [00:00<?, ?it/s]

<ipython-input-17-fa459c92f69e>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm_notebook(dataloader,


Evaluating:   0%|          | 0/11 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/11 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/11 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
print(max(accs_val))

0.7333332896232605


In [ ]:
# hidden_dim = 256, num_layers = 1, aggr_type: max: 0.51
# hidden_dim = 1024, num_layers = 3, aggr_type: mean: 0.33

# hidden_dim = 1024, num_layers = 1, aggr_type: mean 0.52

# hidden_dim = 256, num_layers = 3, aggr_type: mean 0.43

# hidden_dim = 4096, num_layers = 1, aggr_type:mean 0.34

# hidden_dim = 1024, num_layers = 1, aggr_type:mean 0.66

# hidden_dim = 2048, num_layers = 1, vocab_size = 71113, aggr_type: mean 0.469
# hidden_dim = 2048, num_layers = 1, vocab_size = 71113, aggr_type: mean 0.469
# hidden_dim = 1024, num_layers = 1, без дропаута aggr_type:mean 0.60
# hidden_dim = 1024, num_layers = 1, dropout = 0.2 aggr_type:mean 0.66
# hidden_dim = 1024, num_layers = 1, dropout1 = 0.2, dropout2 = 0.2 aggr_type: mean 0.62
# hidden_dim = 1024, num_layers = 1, dropout = 0.3 aggr_type:mean 0.24
# hidden_dim = 1024, num_layers = 1, dropout = 0.2 vocab_size = 13349 aggr_type:mean 0.67
# hidden_dim = 1024, num_layers = 1, dropout = 0.3 vocab_size = 13349 aggr_type: mean 0.58
# gru: hidden_dim = 1024, num_layers = 1, dropout = 0.2 vocab_size = 13349 aggr_type: mean 0.68
# gru: hidden_dim = 1024, num_layers = 2, dropout = 0.2 vocab_size = 13349 aggr_type: mean mean 0.68
# gru: hidden_dim = 1024, num_layers = 2, dropout = 0.2 vocab_size = 18929 aggr_type: mean 0.69
# gru: hidden_dim = 1024, num_layers = 3, dropout = 0.2 vocab_size = 18929 aggr_type: mean mean 0.63
# gru: hidden_dim = 1024, num_layers = 2, dropout = 0.2 vocab_size = 18929 + toch.cat aggr_type: mean 0.60
#gru: hidden_dim = 1024, num_layers = 2, dropout = 0.2 vocab_size = 18929 + lr = 1e-3 : mean 0.718
#gru: hidden_dim = 1024, num_layers = 2, dropout = 0.1 vocab_size = 18929 + lr = 1e-3 : 0.70
#gru: hidden_dim = 1024, num_layers = 1, dropout = 0.1 vocab_size = 18929 + lr = 1e-3 : 0.712
#gru: hidden_dim = 1024, num_layers = 1, dropout = 0.1 vocab_size = 18929 + lr = 1e-3 + num_epochs = 30: 0.73

#rnn: hidden_dim = 1024, num_layers = 1, dropout = 0.1 vocab_size = 18929 + lr = 1e-3 + num_epochs = 30: 0.74
#rnn: hidden_dim = 1024, num_layers = 1, dropout = 0.1 vocab_size = 31329 + lr = 1e-3 + num_epochs = 30: 0.69
#rnn: hidden_dim = 1280, num_layers = 1, dropout = 0.1 vocab_size = 18929 + lr = 1e-3 + num_epochs = 30: 0.706
#rnn: hidden_dim = 900, num_layers = 1, dropout = 0.1 vocab_size = 18929 + lr = 1e-3 + num_epochs = 30: 0.72
#rnn: hidden_dim = 1124, num_layers = 1, dropout = 0.1 vocab_size = 18929 + lr = 1e-3 + num_epochs = 30:
#rnn: hidden_dim = 1024, num_layers = 1, dropout = 0.1 vocab_size = 18929 + lr = 1e-3 + num_epochs = 40: mean 0.73
#rnn: hidden_dim = 1024, num_layers = 1, dropout = 0.1 vocab_size = 18929 + lr = 1e-3 + num_epochs = 45: mean 0.739
#rnn: hidden_dim = 1024, num_layers = 1, dropout = 0.1 vocab_size = 18929 + lr = 1e-3 + num_epochs = 50: mean 0.736
#rnn: hidden_dim = 1024, num_layers = 1, dropout = 0.1 vocab_size = 18929 + lr = 1e-3 + num_epochs = 55: mean 0.718

#gru: hidden_dim = 1024, num_layers = 1, dropout = 0.1 vocab_size = 18929 + lr = 1e-3 + num_epochs = 30: mean 0.69
#rnn: hidden_dim = 1024, num_layers = 1, dropout = 0.1 vocab_size = 18929 + lr = 1e-4 + num_epochs = 30: 0.654
#rnn: hidden_dim = 1024, num_layers = 1, dropout = 0.1 vocab_size = 18929 + lr = 1e-3 + num_epochs = 60: 0.548
#rnn: hidden_dim = 256, num_layers = 1, dropout = 0.1 vocab_size = 18929 + lr = 1e-3 + num_epochs = 20: (mean) 0.57
#rnn: hidden_dim = 512, num_layers = 1, dropout = 0.1 vocab_size = 18929 + lr = 1e-3 + num_epochs = 20: mean 0.68
#rnn: hidden_dim = 512, num_layers = 1, dropout = 0.1 vocab_size = 18929 + lr = 1e-3 + num_epochs = 30: mean 0.70
#rnn: hidden_dim = 1536, num_layers = 1, dropout = 0.1 vocab_size = 18929 + lr = 1e-3 + num_epochs = 30: mean 0.65
#rnn: hidden_dim = 768, num_layers = 1, dropout = 0.1 vocab_size = 18929 + lr = 1e-3 + num_epochs = 30: mean 0.68
#rnn: hidden_dim = 1024, num_layers = 1, dropout = 0.1 vocab_size = 18929 + lr = 1e-3 + num_epochs = 40: mean 0.73
#rnn: hidden_dim = 1024, num_layers = 1, dropout = 0.5 vocab_size = 18929 + lr = 1e-3 + num_epochs = 30: mean 0.69
#rnn: hidden_dim = 1024, num_layers = 1, dropout = 0.6 vocab_size = 18929 + lr = 1e-3 + num_epochs = 30: mean 0.65
#rnn: hidden_dim = 1024, num_layers = 1, dropout = 0.4 vocab_size = 18929 + lr = 1e-3 + num_epochs = 30: mean 0.71
#rnn: hidden_dim = 1024, num_layers = 4, dropout = 0.1 vocab_size = 18929 + lr = 1e-3 + num_epochs = 10: mean 0.35
#gru: hidden_dim = 1024, num_layers = 3, dropout = 0.2 vocab_size = 18929 + lr = 1e-3 : num_epochs = 10: mean 0.56
#gru: hidden_dim = 1024, num_layers = 2, dropout = 0.4 vocab_size = 18929 + lr = 1e-3 : num_epochs = 30: mean 0.718
#gru: hidden_dim = 256, num_layers = 1, dropout = 0.1 vocab_size = 18929 + lr = 1e-3 : num_epochs = 10: mean 0.54
#gru: hidden_dim = 256, num_layers = 2, dropout = 0.1 vocab_size = 18929 + lr = 1e-3 : num_epochs = 10: mean 0.51
#gru: hidden_dim = 256, num_layers = 3, dropout = 0.1 vocab_size = 18929 + lr = 1e-3 : num_epochs = 10: mean 0.57
#gru: hidden_dim = 256, num_layers = 4, dropout = 0.1 vocab_size = 18929 + lr = 1e-3 : num_epochs = 10: mean 0.47 max 0.46
#gru: hidden_dim = 256, num_layers = 5, dropout = 0.1 vocab_size = 18929 + lr = 1e-3 : num_epochs = 10: mean mean 0.41
#gru: hidden_dim = 256, num_layers = 6, dropout = 0.1 vocab_size = 18929 + lr = 1e-3 : num_epochs = 10: mean 0.41, max 0.38
#gru: hidden_dim = 256, num_layers = 7, dropout = 0.1 vocab_size = 18929 + lr = 1e-3 : num_epochs = 10: mean 0.42
#gru: hidden_dim = 256, num_layers = 10, dropout = 0.1 vocab_size = 18929 + lr = 1e-3 : num_epochs = 10: mean 0.40
#gru: hidden_dim = 256, num_layers = 20, dropout = 0.1 vocab_size = 18929 + lr = 1e-3 : num_epochs = 10: mean 0.31


In [ ]:
for aggr_type in accs_val_type:
  print(aggr_type, max(accs_val_type[aggr_type]))

max 0.3272727131843567
mean 0.4060606062412262
last 0.290909081697464


Ваша задача — обучить RNN-модель на тренировочных данных и получить максимальное возможное accuracy на тестовой части данных. За основу можно взять код с занятия. **Обратите внимание, что здесь у нас задача классификации на 8 классов, а не на 2, как было на занятии.**

Чтобы улучшить качество базовой модели, можно попробовать различные идеи экспериментов. Вот несколько идей:
* **Модель RNN**. Попробуйте LSTM и GRU. Мы советуем обратить внимание на [GRU](https://pytorch.org/docs/stable/generated/torch.nn.GRU.html), так как интерфейс этого класса ничем не отличается от обычной Vanilla RNN, которую мы использовали на семинаре.
* **Увеличение количества рекуррентных слоев модели**. Это можно сделать с помощью параметра `num_layers` в классе `nn.RNN`. В такой модели выходы первой RNN передаются в качестве входов второй RNN и так далее.
* **Изменение вида агрреграции выхода RNN**. В семинаре мы пробовали аггрегировать выходы RNN с помощью max, mean и получения последнего выхода. Можно попробовать другие варианты. Например, конкатенировать результат агрегации и эмбеддинг с последнего токена.
* **Подбор гиперпараметров и обучение до сходимости**. Можно, например, увеличить количество эпох обучения нейросети, а также попробовать различные гиперпараметры: размер словаря, `dropout_rate`, `hidden_dim`.

## Сдача задания

Ниже приведена функция, которую вам необходимо запустить для обученной модели, чтобы получить предсказания на тестовой выборке. Здесь model — ваша обученная модель, dataloader — test_dataloader, построенный на основе тестовой части данных (dataset['test']):

In [ ]:

# model = SimpleRNN(hidden_dim=hidden_dim,
#                   vocab_size=len(vocab),
#                   num_classes=8,
#                   aggregation_type='mean'
#                   ).to(device)
# criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
# optimizer = torch.optim.Adam(model.parameters())

def get_predictions(model, dataloader):
    """
    Calculate accuracy on data from dataloader.
    """

    model.eval()
    predictions = []
    with torch.no_grad():
        for batch in tqdm_notebook(dataloader,
                                   desc=f'Evaluating'):
            logits = model(batch['input_ids'])
            predictions.append(logits.argmax(dim=1))

    predictions = torch.cat(predictions).data.cpu().numpy()

    return predictions

In [ ]:
predictions = get_predictions(model, test_dataloader)
predictions = [label_to_writers[x] for x in predictions]
predictions

<ipython-input-37-def29f4174b2>:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm_notebook(dataloader,


Evaluating:   0%|          | 0/13 [00:00<?, ?it/s]

['Pratchett',
 'Remark',
 'King',
 'Pratchett',
 'Akunin',
 'Akunin',
 'Akunin',
 'Pratchett',
 'Akunin',
 'Bulychev',
 'Bulychev',
 'Bulychev',
 'Akunin',
 'Bulychev',
 'Akunin',
 'Akunin',
 'Akunin',
 'Bulychev',
 'King',
 'Chehov',
 'Akunin',
 'Bulychev',
 'Remark',
 'Remark',
 'Remark',
 'King',
 'Akunin',
 'Pratchett',
 'Bulychev',
 'Akunin',
 'Chehov',
 'Chehov',
 'King',
 'Akunin',
 'Bulychev',
 'Bulychev',
 'Bulychev',
 'Bulychev',
 'Bulychev',
 'Akunin',
 'Gogol',
 'King',
 'Bulychev',
 'Akunin',
 'Chehov',
 'Dostoevsky',
 'Bulychev',
 'Bulychev',
 'Akunin',
 'Akunin',
 'Remark',
 'King',
 'Akunin',
 'Bulychev',
 'King',
 'Akunin',
 'Chehov',
 'Remark',
 'Akunin',
 'Remark',
 'Chehov',
 'King',
 'King',
 'Akunin',
 'Akunin',
 'Akunin',
 'Chehov',
 'Pratchett',
 'Pratchett',
 'Akunin',
 'Akunin',
 'Akunin',
 'Remark',
 'Bulychev',
 'Akunin',
 'King',
 'King',
 'King',
 'Akunin',
 'Bulychev',
 'King',
 'Bulychev',
 'Pratchett',
 'Gogol',
 'King',
 'Gogol',
 'Akunin',
 'King',
 '

In [ ]:
np.save('submission_hw07.npy', predictions, allow_pickle=True)
print('Ответ сохранен в файл `submission_hw07.npy`')

Ответ сохранен в файл `submission_hw07.npy`
